In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import requests
import plotly.express as px
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()  # загружаются переменные из файла

DATE_BEGIN = os.getenv('DATE_BEGIN')
DATE_END = os.getenv('DATE_END')
API_URL = os.getenv('API_URL')

resp = requests.get(f'{API_URL}/registrations', params={'begin': DATE_BEGIN, 'end': DATE_END})
resp2 = requests.get(f'{API_URL}/visits', params={'begin': DATE_BEGIN, 'end': DATE_END})
df_visits2 = pd.DataFrame(resp2.json())
df_regs2 = pd.DataFrame(resp.json())

In [ ]:
visits3=df_visits2.copy()
regs3=df_regs2.copy()

In [ ]:
visits4=visits3[(visits3['platform']!='bot')&(~visits3['user_agent'].str.contains('bot'))]
visits4['datetime']=pd.to_datetime(visits4['datetime'])
visits_final=visits4.sort_values(by=['visit_id', 'datetime']).drop_duplicates(subset='visit_id', keep='last')
visits_final['date_group']=visits_final['datetime'].dt.date
visits_4_join=visits_final.groupby(['date_group', 'platform'], as_index=False)['visit_id'].count()

In [ ]:
regs4=regs3.drop_duplicates()
regs4['date_group']=pd.to_datetime(regs4['datetime']).dt.date
regs_4_join=regs4.groupby(['date_group', 'platform'], as_index=False)['user_id'].count()

In [ ]:
join_df=pd.merge(visits_4_join, regs_4_join, on=['date_group', 'platform'], how='inner')
join_df['conversion']=join_df['user_id']/join_df['visit_id']*100
join_df.rename(columns={'visit_id': 'visits', 'user_id': 'registrations'}, inplace=True)
join_df.to_json('./conversion.json')

In [ ]:
ads = pd.read_csv('./ads.csv')

In [ ]:
ads['date']=pd.to_datetime(ads['date'])
ads1=ads.copy()
ads1['date_group']=ads['date'].dt.date
ads2=ads1[['date_group', 'cost', 'utm_campaign']]

In [ ]:
join_df_clone = join_df.copy()
join_df_clone_grouped=join_df_clone.groupby(['date_group'], as_index=False)[['visits', 'registrations']].sum()
join_df_clone_grouped_ads2=pd.merge(join_df_clone_grouped, ads2, on=['date_group'], how='left')
join_df_clone_grouped_ads2[join_df_clone_grouped_ads2[ads2.columns.difference(['date_group'])].isna().any(axis=1)].count()
join_df_clone_grouped_ads2['cost'] = join_df_clone_grouped_ads2['cost'].fillna(0).astype(int)
join_df_clone_grouped_ads2['utm_campaign'] = join_df_clone_grouped_ads2['utm_campaign'].fillna('none')
join_df_final=join_df_clone_grouped_ads2.sort_values('date_group')
join_df_final.to_json('./ads.json')

In [ ]:
os.makedirs('./charts', exist_ok=True)

In [ ]:
grouped1 = join_df.groupby('date_group', as_index=False)['visits'].sum()
plt.figure(figsize=(20, 8))
plt.title("Total visits", fontsize=16)
bars=plt.bar(grouped1['date_group'], grouped1['visits'], color='#9FC7DD')
plt.grid(visible=True, axis='y')
plt.xticks(grouped1['date_group'], rotation=45, ha="right", fontsize=4) 
plt.xlabel("date_group", fontsize=16)
plt.ylabel("visits", fontsize=16)
plt.bar_label(bars, fontsize=6)
plt.savefig('./charts/total_visits.png')
plt.show()

In [ ]:
plt.figure(figsize=(20, 8))
plt.title("Visits by Platform (Stacked)", fontsize=16)
bottom=np.zeros(join_df['date_group'].nunique())
for i in join_df['platform'].unique():
    join_df1=join_df[join_df['platform'] == i]
    plt.bar(join_df1['date_group'], join_df1['visits'], label=i, bottom=bottom)
    bottom += np.array(join_df1['visits'])
    print(bottom)
plt.legend(title="platform")
plt.xticks(join_df['date_group'], rotation=45, ha="right", fontsize=4) 
plt.grid(visible=True, axis='both')
plt.xticks(rotation=45, ha="right") 
plt.xlabel("date_group", fontsize=16)
plt.savefig('./charts/total_visits_by_platform.png')
plt.show()

In [ ]:
grouped2 = join_df.groupby('date_group', as_index=False)['registrations'].sum()
plt.figure(figsize=(20, 8))
plt.title("Total Weekly Registrations", fontsize=16)
bars=plt.bar(grouped2['date_group'], grouped2['registrations'], color='#9FC7DD')
plt.grid(visible=True, axis='y')
plt.xticks(grouped2['date_group'], rotation=45, ha="right", fontsize=4) 
plt.xlabel("date_group", fontsize=16)
plt.ylabel("registrations", fontsize=16)
plt.bar_label(bars, fontsize=6)
plt.savefig('./charts/total_registrations.png')
plt.show()

In [ ]:
plt.figure(figsize=(20, 8))
plt.title("Weekly Registrations by Platform (Stacked)", fontsize=16)
bottom=np.zeros(join_df['date_group'].nunique())
for i in join_df['platform'].unique():
    join_df1=join_df[join_df['platform'] == i]
    plt.bar(join_df1['date_group'], join_df1['registrations'], label=i, bottom=bottom)
    bottom += np.array(join_df1['registrations'])
    print(bottom)
plt.legend(title="platform")
plt.xticks(join_df['date_group'], rotation=45, ha="right", fontsize=4) 
plt.grid(visible=True, axis='both')
plt.xticks(rotation=45, ha="right") 
plt.xlabel("date_group", fontsize=16)
plt.savefig('./charts/total_registrations_by_platform.png')
plt.show()

In [ ]:
grouped3=join_df.groupby(['date_group'], as_index=False)[['visits', 'registrations']].sum()
grouped3['conversion']=((grouped3['registrations']/grouped3['visits'])*100).round(0).astype(int)
plt.figure(figsize=(20, 8))
plt.title("Overall Conversion", fontsize=16)
graph=plt.plot(grouped3['date_group'], grouped3['conversion'], data=grouped3, marker='.', label='Общая конверсия')
plt.grid(visible=True, axis='both')
plt.xticks (grouped3['date_group'], rotation=45, ha="right", fontsize=4) 
plt.xlabel("Date", fontsize=16)
plt.ylabel("Conversion (%)", fontsize=16)
plt.legend()
for i, j in zip(grouped3['date_group'], grouped3['conversion']):
    plt.text(i, j, str(j) + "%")
plt.savefig('./charts/overall_conversion.png')
plt.show()

In [ ]:
join_df_prcn=join_df.copy()
join_df_prcn['conversion']=(join_df_prcn['conversion']*100).round(0).astype(int)
fig, axes = plt.subplots(3, 1)
fig.set_size_inches(20, 16)

axes[0].plot(
    join_df_prcn[join_df_prcn['platform'] == 'android']['date_group'],
    join_df_prcn[join_df_prcn['platform'] == 'android']['conversion'], 
    label='android', 
    marker="."
)
axes[0].set_title('Conversion android')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Conversion (%)')
for x, y in zip(
    join_df_prcn[join_df_prcn['platform'] == 'android']['date_group'],
    join_df_prcn[join_df_prcn['platform'] == 'android']['conversion']
):
    axes[0].text(x, y, str(y)+"%", fontsize=8)
axes[0].grid()
axes[0].legend(loc='lower left')

axes[1].plot(
    join_df_prcn[join_df_prcn['platform'] == 'ios']['date_group'],
    join_df_prcn[join_df_prcn['platform'] == 'ios']['conversion'], 
    label='ios', 
    marker="."
)
axes[1].set_title('Conversion ios')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Conversion (%)')
for x, y in zip(
    join_df_prcn[join_df_prcn['platform'] == 'ios']['date_group'],
    join_df_prcn[join_df_prcn['platform'] == 'ios']['conversion']
):
    axes[1].text(x, y, str(y)+"%", fontsize=8)
axes[1].grid()
axes[1].legend(loc='upper right')

axes[2].plot(
    join_df_prcn[join_df_prcn['platform'] == 'web']['date_group'], 
    join_df_prcn[join_df_prcn['platform'] == 'web']['conversion'], 
    label='web', 
    marker="."
)
axes[2].set_title('Conversion ios')
axes[2].set_xlabel('Date')
axes[2].set_ylabel('Conversion (%)')
for x, y in zip(
    join_df_prcn[join_df_prcn['platform'] == 'web']['date_group'],
    join_df_prcn[join_df_prcn['platform'] == 'web']['conversion']
):
    axes[2].text(x, y, str(y)+"%", fontsize=8)
axes[2].grid()
axes[2].legend(loc='upper right')

plt.savefig('./charts/overall_conversion_by_platfotm.png')

In [ ]:
cost_plot=join_df_final.copy()
plt.figure(figsize=(20, 8))
plt.title("Aggregated Ad Campaign Costs (by day)", fontsize=16)
graph1=plt.plot(cost_plot['date_group'], cost_plot['cost'], data=cost_plot, marker='.')
plt.grid(visible=True, axis='both')
plt.xticks (cost_plot['date_group'], rotation=45, ha="right", fontsize=4) 
plt.xlabel("Date", fontsize=16)
plt.ylabel("Cost (RUB)", fontsize=16)
for i, j in zip(cost_plot['date_group'], cost_plot['cost']):
    plt.text(i, j, str(j) + "RUB")
plt.savefig('./charts/agg_ad_campaign_costs.png')
plt.show()

In [ ]:
comparison=join_df_final.copy()
fig, axes = plt.subplots(2, 1)
fig.set_size_inches(20, 12)

colors=['b', 'g', 'r', 'c', 'm', 'y']

mean_visits=np.mean(comparison['visits'])
axes[0].plot(comparison['date_group'], comparison['visits'], marker=".", label='Visits', color='k')
axes[0].axhline(y=mean_visits, linestyle='--', label='Average Number of Visits', color='gray')
for num, group in comparison.groupby(
    (comparison['utm_campaign'] != comparison['utm_campaign'].shift()).cumsum()):
    campaign_group = group['utm_campaign'].iloc[0]
    if campaign_group != 'none':
        axes[0].axvspan(
            group['date_group'].iloc[0],
            group['date_group'].iloc[-1],
            alpha=0.3,
            color=colors[num],
            label=campaign_group
        )
axes[0].set_title('Visits during marketing active days')
axes[0].set_ylabel('Unique Visits')
axes[0].grid()
axes[0].legend(loc='lower left')
axes[0].tick_params(axis='x', rotation=45)

mean_regs=np.mean(comparison['registrations'])
axes[1].plot(comparison['date_group'], comparison['registrations'], marker=".", label='Registrations', color='g')
axes[1].axhline(y=mean_regs, linestyle='--', label='Average Number of Registrations', color='gray')
for num, group in comparison.groupby(
    (comparison['utm_campaign'] != comparison['utm_campaign'].shift()).cumsum()):
    campaign_group = group['utm_campaign'].iloc[0]
    if campaign_group != 'none':
        axes[1].axvspan(
            group['date_group'].iloc[0],
            group['date_group'].iloc[-1],
            alpha=0.3,
            color=colors[num],
            label=campaign_group
        )
axes[1].set_title('Registrations during marketing active days')
axes[1].set_ylabel('Unique Registrations')
axes[1].grid()
axes[1].legend(loc='upper right')
axes[1].tick_params(axis='x', rotation=45)

plt.savefig('./charts/visits_regs_by_campaigns_colored_zones.png')